In [14]:
a=dict()
for i in range(5):
    a[i]={i+j:0 for j in range(5)}
print(a)
print(a[2].keys())

{0: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}, 1: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}, 2: {2: 0, 3: 0, 4: 0, 5: 0, 6: 0}, 3: {3: 0, 4: 0, 5: 0, 6: 0, 7: 0}, 4: {4: 0, 5: 0, 6: 0, 7: 0, 8: 0}}
dict_keys([2, 3, 4, 5, 6])


In [6]:
import math
import numpy as np
query_list_path = "q_100_d_10000/query_list.txt"
doc_list_path = "q_100_d_10000/doc_list.txt"

with open(query_list_path, "r") as f:
    q_list = f.read().split('\n')[:-1]
with open(doc_list_path, "r") as f:
    d_list = f.read().split('\n')[:-1]
d_dict = {doc:doc_index for doc_index,doc in enumerate(d_list)}

def get_query_word(q):
    with open("q_100_d_10000/queries/{}.txt".format(q),'r') as f:
        words = f.read().split(' ')
    return words
def get_doc_word(d):
    with open("q_100_d_10000/docs/{}.txt".format(d),'r') as f:
        words = f.read().split(' ')
    return words
def get_random_probability_matrix(event_num,condition_num):
    matrix=np.random.random_sample((event_num,condition_num))
    for i in range(condition_num):
        temp_sum = matrix[:,i].sum()
        matrix[:,i]/=temp_sum #set sum to 1
    return matrix

class parameter_retriever:
    def __init__(self,topic_num):
        self.index_term_dict=dict()
        self.index_term_num = 0
        self.topic_num = topic_num
        self.doc_num = len(d_list)

        self.c_wd = [dict()]*self.index_term_num
        self.doc_length = [0]*len(d_list)
        self.c_w = [0]*self.index_term_num

        self.P_w_T = []
        self.P_T_d = []
        self.sum_of_cP_in_D=[]
        self.sum_of_cP_in_V=[]


    def create_index_term_set(self):
        print("creating index term set")
        print(" creating index term set from query")
        index_term_set_q = set()
        for q in q_list:
            words = get_query_word(q)
            index_term_set_q = index_term_set_q.union(set(words))

        print(" creating index term set from doc")
        index_term_set_d = set()
        for d in d_list:
            words = get_doc_word(d)
            index_term_set_d = index_term_set_d.union(set(words))

        index_term_set = index_term_set_q.union(index_term_set_d)
        self.index_term_dict={index_term:index_term_index for index_term_index,index_term in enumerate(index_term_set)}
        self.index_term_num = len(index_term_set)
        print("number of words in index_term_set: {}".format(self.index_term_num))
        print("...done")

    def get_word_count_in_doc(self):
        print("getting word counts in doc/doc_length/BG")
        self.c_wd = [dict()]*self.index_term_num
        self.doc_length = [0]*len(d_list)
        self.c_w = [0]*self.index_term_num

        for doc_index,doc in enumerate(d_list):
            words = get_doc_word(doc)
            for word in set(words):
                if word in self.index_term_dict:
                    self.c_w[self.index_term_dict[word]]+=1
                self.c_wd[self.index_term_dict[word]][doc_index]=0
            for word in words:
                if word in self.index_term_dict:
                    self.c_wd[self.index_term_dict[word]][doc_index]+=1
                    self.doc_length[doc_index]+=1
        print("...done")

    def initPossibilities(self):
        print("initializing possibilities")
        random_matrix = get_random_probability_matrix(self.index_term_num,self.topic_num)
        for index_term_idx in range(self.index_term_num):
            self.P_w_T.append([probability for probability in random_matrix[index_term_idx,:].tolist()])
        
        random_matrix = get_random_probability_matrix(self.topic_num,self.doc_num)
        for topic_idx in range(self.topic_num):
            self.P_T_d.append([probability for probability in random_matrix[topic_idx,:].tolist()])
        print("...done")

    def calculate_P_T_wd(self,topic_idx,index_term_index,doc_index):
        buffer = self.P_w_T[index_term_index][topic_idx]*self.P_T_d[topic_idx][doc_index]
        sum=0
        for k in range(self.topic_num):
            sum+=self.P_w_T[index_term_index][k]*self.P_T_d[k][doc_index]
        return buffer/sum

    def E_step(self):
        print("start E_step")
        self.sum_of_cP_in_D=[[0]*self.topic_num]*self.index_term_num
        self.sum_of_cP_in_V=[[0]*len(d_list)]*self.topic_num
        for index_term_index in range(self.index_term_num):
            for topic_idx in range(self.topic_num):
                for doc_index in self.c_wd[index_term_index].keys():
                    cP=self.c_wd[index_term_index][doc_index]*self.calculate_P_T_wd(topic_idx,index_term_index,doc_index)
                    self.sum_of_cP_in_D[index_term_index][topic_idx]+=cP
                    self.sum_of_cP_in_V[topic_idx][doc_index]+=cP
        print("...done")

    def M_step(self):
        print("start M_step")
        print(" process P_w_T")
        for topic_index in range(self.topic_num):
            sum_of_topic_k = 0
            for index_term_index in range(self.index_term_num):
                sum_of_topic_k+=self.sum_of_cP_in_D[index_term_index][topic_index]
            for index_term_index in range(self.index_term_num):
                self.P_w_T[index_term_index][topic_index]=self.sum_of_cP_in_D[index_term_index][topic_index]/sum_of_topic_k
        print(" process P_T_d")
        for doc_index in range(len(d_list)):
            if self.doc_length[doc_index]==0:
                for topic_index in range(self.topic_num):
                    self.P_T_d[topic_index][doc_index] = 1/self.topic_num
            else:
                for topic_index in range(self.topic_num):
                    self.P_T_d[topic_index][doc_index]=self.sum_of_cP_in_V[topic_index][doc_index]/self.doc_length[doc_index]
        print("...done")

    def iter(self):
        self.E_step()
        self.M_step()



class PLSA:
    def __init__(self,topic_num,alpha,beta):
        self.topic_num=topic_num
        self.param = parameter_retriever(topic_num)
        
        self.alpha=alpha
        self.beta=beta
    
    def init_param(self):
        self.param.create_index_term_set()
        self.param.get_word_count_in_doc()
        self.param.initPossibilities()

    def train(self,n):
        for i in range(n):
            print("iter: {}".format(i))
            self.param.iter()

    def get_sim(self,doc,q):
        if self.param.doc_length[doc]==0:
            return -999999999
        logsum=0
        for word in get_query_word(q):
            if word in self.param.index_term_set:
                first = self.alpha*self.param.c_wd[word][doc]/self.param.doc_length[doc]

                second = 0
                for topic_idx in range(self.topic_num):
                    second+=self.param.P_w_T[word][topic_idx]*self.param.P_T_d[topic_idx][doc]
                second*=self.beta
                
                third = (1-self.alpha-self.beta)*self.param.c_w[word]/self.param.doc_num

                temp = np.logaddexp(first, second)
                temp = np.logaddexp(temp, third)
                logsum+=temp
        return logsum
    def query(self,q):
        sim={}
        for idx_doc,doc in enumerate(d_list):

            sim[doc] = self.get_sim(idx_doc,q)
        sim = sorted(sim.items(), key=lambda x:x[1],reverse=True)
        ans = ""
        for i in sim:
            ans+=i[0]+' '
        return ans


In [7]:
#time: <10sec
plsa=PLSA(32,0.3,0.1)
plsa.init_param()

creating index term set
 creating index term set from query
 creating index term set from doc
number of words in index_term_set: 234
...done
initializing possibilities
...done
getting word counts in doc/doc_length/BG
...done


In [9]:
#Estep<15sec, Mstep<1min30sec
#total<2min45sec
plsa.train(1)

iter: {} 0
start E_step
...done
start M_step
 process P_w_T
 process P_T_d
...done


In [ ]:
#time: <2min
f = open("ans.txt","w")
f.write("Query,RetrievedDocuments\n")
for q in q_list:
    ranking=plsa.query(q)
    f.writelines(q+","+ranking+'\n')
f.close()

In [8]:
import pickle
with open('init.pickle', 'wb') as f:
    pickle.dump(plsa, f)